In [1]:
import os
import json
#Tokenize text
from konlpy.tag import Kkma 
from konlpy.utils import pprint 
import pandas as pd
import numpy as np

In [18]:


class ParseText:
    def __init__(self):
        self.dir=""
        self.stopwords=pd.DataFrame(columns=['vocab','pos'])
        self.data=[]
        self.parsedData=[]
        self.kkma=None
        
    def openAllJsonTest(self,dirname,limit=0): #"./2020-02-019.도서자료요약_Sample"
        self.dir+=dirname## get path
        #1) open json file
        jsonobj=[]
        
        for file in os.listdir(dirname):
            if limit<0:
                break
            jsonfd=open(dirname+"/"+file, encoding="utf-8")## get file descriptor
            jsonstr=jsonfd.read()## get str
            jsonobj.append(jsonstr)
            limit-=1
        
        return jsonobj
    def openAllJson(self,dirname): #"./2020-02-019.도서자료요약_Sample"
        self.dir+=dirname## get path
        #1) open json file
        jsonobj=[]
        
        for file in os.listdir(dirname):
 
            jsonfd=open(dirname+"/"+file, encoding="utf-8")## get file descriptor
            jsonstr=jsonfd.read()## get str
            jsonobj.append(jsonstr)

        
        return jsonobj
    
    def organizeFile(self,jsonobj):
		#2) select passages and summaries
        file=json.loads(jsonobj)
        return {'id':file["passage_id"],'passage':file["passage"],'summary':file["summary"]}
    
    def organizeFiles(self,dirname):
        jsonobj=self.openAllJson(dirname)
        for i in range(len(jsonobj)):
            self.data.append(self.organizeFile(jsonobj[i]))
    def organizeFilesTest(self,dirname,limit=10):
            jsonobj=self.openAllJsonTest(dirname,limit=limit)
            for i in range(len(jsonobj)):
                self.data.append(self.organizeFile(jsonobj[i]))
        
    
    def setStopwords(self,path):#stopwords.txt
        self.kkma=Kkma()
        f=open(path,encoding="utf-8")
        self.stopwords=pd.concat((self.stopwords,pd.DataFrame(self.kkma.pos(f.read()),columns=['vocab','pos'])),axis=0)
    
    def preprocessFile(self,data):
        kkma=Kkma()
        passage=data['passage']
        word_dict=pd.DataFrame(columns=['vocab','pos','mundan_id','sent_id','eojeol_id'])
        for i,mundan in enumerate(passage.split('\n')): #i는 n 번째 문단이라는 뜻
            for j, sent in enumerate(mundan.split('.')):#j 는 n번째 문장이라는 뜻
                for k, word in enumerate(sent.split(' ')):#k는 n번째 어절이라는 뜻 
                    passage_hs=kkma.pos(word) #list in list
                    for hs in passage_hs: #어절 내부의 위치는 기록하지 않는다. 어절은 통째로 뺄 것이기 때문이다. 
                        position=[i,j,k]+list(hs)
                        current=np.reshape(np.array(position),(1,-1))
                        current=pd.DataFrame(current,columns=['mundan_id','sent_id','eojeol_id','vocab','pos'])
                        word_dict=pd.concat((word_dict,current),axis=0)

                

        return word_dict


    def preprocessFiles(self,stoppath,stop=0):
        if stop==0:
            self.setStopwords(stoppath)
        #new passage object with words-->words & PoS tuples / only NNG, VV
        for d in self.data:
            word_PoS_pos=self.preprocessFile(d)
            #품사,stopwords 필터, 재정렬
            filtered=word_PoS_pos[(word_PoS_pos.pos=='NNG')|(word_PoS_pos.pos=='VV')&(~word_PoS_pos['vocab'].isin(list(self.stopwords.vocab)))].reset_index()
            self.parsedData.append(filtered)
        return self.parsedData
    
    def doAll(self,type=0,dr=None,stop=None):
        if type==0:
            dirname=input('dirname of jsonobj directory')
            stoppath=input('dirname of stopwords txt')
        else:
            dirname=dr
            stoppath=stop
        self.organizeFiles(dirname)
        data_stack=self.preprocessFiles(stoppath)
        return data_stack
    def doAllTest(self,dirname,stoppath,limit):

        self.organizeFilesTest(dirname,limit=limit)
        data_stack=self.preprocessFiles(stoppath)
        return data_stack



In [20]:
parser2=ParseText()
result=parser2.doAllTest("./2020-02-019.도서자료요약_Sample","stopwords.txt",limit=10)

In [21]:
result[:5]

[    index vocab  pos mundan_id sent_id eojeol_id
 0       0    선거  NNG         0       0         1
 1       0    개혁  NNG         0       0         1
 2       0    유사  NNG         0       0         2
 3       0    문제  NNG         0       0         3
 4       0   유권자  NNG         0       1         1
 ..    ...   ...  ...       ...     ...       ...
 81      0    참여  NNG         0       4         6
 82      0    확대  NNG         0       4         7
 83      0     전  NNG         0       4         8
 84      0   나아지   VV         0       4         9
 85      0    한계  NNG         0       4        13
 
 [86 rows x 6 columns],
     index vocab  pos mundan_id sent_id eojeol_id
 0       0   종교가  NNG         0       0         1
 1       0    인간  NNG         0       0         2
 2       0     꿈  NNG         0       0         3
 3       0    실현  NNG         0       0         4
 4       0    해답  NNG         0       0         5
 ..    ...   ...  ...       ...     ...       ...
 75      0   종교가  NNG   

In [ ]:
class TextRank():
    def __init__(self,data):
        self.nodes=data
        self.edges=[]
    def makeEdge(self, window=5):
        #for all passages
        for df in self.nodes:
            #list 
            #for all window size
            for i in range(1,window+1):
                for index in range(0,df.shape[0]-i):
                    # for all words in passages
                    self.edges.append((df.loc[index,'vocab'],df.loc[index+i,'vocab']))
    return edges
    def flip(self):
        processed_edges=edges
        edges_set=[]
        for i,edge in enumerate(edges):
            if edge not in edges_set:
                if edge[::-1] not in edges_set:
                    edges_set.append(edge)
                else:
                    processed_edges.pop(i)
                    processed_edges.insert(i,edge[::-1])
        return edges,edges_set
    
    def connectEdge(self):
        vocab=passage_w_pos[0]['vocab']
        vocab=vocab.drop_duplicates()
        vocab=vocab.reset_index()
        vocab.drop('index',inplace=True,axis=1)

        print(vocab.size)
        #2) make co relationship matrx of vocab*vocab
        rel_matrix=np.zeros((vocab.size,vocab.size))

        for pair in counter:
            w1,w2=pair
            #한 integer가 나올 것이다. 
            p1=vocab.loc[vocab['vocab']==w1].index.tolist()
            p2=vocab.loc[vocab['vocab']==w2].index.tolist()
            #3) use counter to fill in the matrix
            count=counter[pair]
            rel_matrix[p1[0]][p2[0]]+=count

            

        #4) normalize matrix only the rows
            # if directed, omit "reduce_flip_edges" stage and make the matrix accordingly

        norm=np.linalg.norm(rel_matrix)
        rel_matrix=rel_matrix/norm
        
        #5) make iterative calculation starting with dot of n*n   with [1....1] (n*1 size)
        iteration=10
        im_vec=np.ones(rel_matrix.shape[0])
        for i in range(iteration):
    im_vec=np.dot(rel_matrix,im_vec)
    
    def returnBest(self,best_count):
        pass